In [5]:
import pandas as pd

**General**
- advanced
- basic
- defense
- scoring
- usage


In [9]:
data = pd.read_excel('AdvancedStats.xlsx')

In [10]:
data.head()

,Unnamed: 0,PLAYER,TEAM,AGE,GP,W,L,MIN,OFFRTG,DEFRTG,...,OREB%,DREB%,REB%,TO Ratio,eFG%,TS%,USG%,PACE,PIE,POSS
0,0,Anthony Edwards,MIN,19,72,23,49,32.1,110.6,115.6,...,2.3,12.1,7.0,9.4,48.8,52.3,26.4,102.71,9.4,"4,961"
1,1,Bojan Bogdanovic,UTA,32,72,52,20,30.8,116.8,107.3,...,1.8,9.9,6.0,9.8,53.6,58.8,22.5,99.94,9.4,"4,604"
2,2,Duncan Robinson,MIA,27,72,40,32,31.4,110.6,108.3,...,0.3,10.9,5.7,8.5,61.4,62.8,16.2,97.55,7.5,"4,602"
3,3,Dwayne Bacon,ORL,25,72,21,51,25.7,101.3,112.8,...,1.5,10.2,5.6,4.9,44.3,49.0,19.3,100.39,6.9,"3,875"
4,4,Enes Freedom,POR,29,72,42,30,24.4,119.7,114.8,...,14.8,28.1,21.3,9.8,60.5,63.6,16.9,98.79,14.1,"3,607"


In [12]:
basicData = pd.read_excel('BasicStats.xlsx')

In [13]:
basicData.head()

,Unnamed: 0,PLAYER,TEAM,AGE,GP,W,L,MIN,PTS,FGM,...,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-
0,0,Stephen Curry,GSW,33,63,37,26,34.2,32.0,10.4,...,5.5,5.8,3.4,1.2,0.1,1.9,47.9,8,0,3.9
1,1,Bradley Beal,WAS,28,60,32,28,35.8,31.3,11.2,...,4.7,4.4,3.1,1.2,0.4,2.3,45.0,4,0,-0.1
2,2,Damian Lillard,POR,30,67,39,28,35.8,28.8,9.0,...,4.2,7.5,3.0,0.9,0.3,1.5,45.7,16,0,3.0
3,3,Joel Embiid,PHI,27,51,39,12,31.1,28.5,9.0,...,10.6,2.8,3.1,1.0,1.4,2.4,49.3,31,0,7.9
4,4,Giannis Antetokounmpo,MIL,26,61,40,21,33.0,28.1,10.3,...,11.0,5.9,3.4,1.2,1.2,2.8,53.9,41,7,6.7
